ASSIGNMENT 2

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizing Fashion-MNIST grayscale images
])

# Load Fashion-MNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:01<00:00, 18.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 341kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 6.36MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.70MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 1 channel for grayscale
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Adjusted for 28x28 input size
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN()


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/10, Loss: 0.2790
Epoch 2/10, Loss: 0.2602
Epoch 3/10, Loss: 0.2372
Epoch 4/10, Loss: 0.2184
Epoch 5/10, Loss: 0.2050
Epoch 6/10, Loss: 0.1908
Epoch 7/10, Loss: 0.1781
Epoch 8/10, Loss: 0.1714
Epoch 9/10, Loss: 0.1603
Epoch 10/10, Loss: 0.1510


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 92.18%


In [ ]:
from torchvision.models import resnet18

# Load pretrained ResNet model
pretrained_model = resnet18(pretrained=True)
pretrained_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)  # Adjusted for grayscale input
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)
pretrained_model.to(device)

# Define optimizer and loss for pretrained model
pretrained_optimizer = optim.Adam(pretrained_model.parameters(), lr=0.001)
pretrained_criterion = nn.CrossEntropyLoss()

# Fine-tuning loop
for epoch in range(num_epochs):
    pretrained_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        pretrained_optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = pretrained_criterion(outputs, labels)
        loss.backward()
        pretrained_optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Pretrained Model Loss: {running_loss/len(train_loader):.4f}")

# Evaluate pretrained model
pretrained_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = pretrained_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

pretrained_accuracy = 100 * correct / total
print(f"Pretrained Model Test Accuracy: {pretrained_accuracy:.2f}%")


Epoch 1/10, Pretrained Model Loss: 0.4949
Epoch 2/10, Pretrained Model Loss: 0.3330
Epoch 3/10, Pretrained Model Loss: 0.2877
Epoch 4/10, Pretrained Model Loss: 0.2594
Epoch 5/10, Pretrained Model Loss: 0.2299
Epoch 6/10, Pretrained Model Loss: 0.2109
Epoch 7/10, Pretrained Model Loss: 0.1898
Epoch 8/10, Pretrained Model Loss: 0.1748
Epoch 9/10, Pretrained Model Loss: 0.1540
Epoch 10/10, Pretrained Model Loss: 0.1565
Pretrained Model Test Accuracy: 90.99%
